In [201]:
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, clear_output
import numpy as np
import pandas as pd
import gmaps
import googlemaps
import os
# print(os.getcwd()) # es la carpeta jupyter
import sys
sys.path.append('../../') # secret.py lo cargará de la raíz del proyecto Data-Science
import secret
gmaps.configure(api_key=secret.GMAPS_KEY) # Your Google API key
#gmaps = googlemaps.Client(key=secret.GMAPS_KEY)

In [153]:
ruta_datasets = '../datasets/'
limit = 10 # numero de registros a leer, -1 para todos

fields_to_drop = ['host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'calculated_host_listings_count',
                  'availability_365', 'price', 'number_of_reviews', 'last_review', 'reviews_per_month']
listing = pd.read_csv(ruta_datasets + 'listings.csv').head(limit).drop(fields_to_drop, axis=1)

fields_to_drop = ['summary', 'space', 'description', 'neighborhood_overview', 'transit', 'access', 'interaction',
                  'name', 'notes', 'house_rules', 'host_about', 'host_verifications', 'latitude', 'longitude',
                  'room_type', 'minimum_nights']
listing_detail = pd.read_csv(ruta_datasets + 'listings.csv.gz',
                             dtype={'zipcode': str}).head(limit).drop(fields_to_drop, axis=1)

lista_total = pd.merge(listing, listing_detail, on='id')

info_box_template = """
<dl>
<dt>ID: {id}</dt><dd>{name}</dd>
<dt>Precio</dt><dd>{price}</dd>
<dt>Tipo de propiedad y habitación</dt><dd>{property_type}, {room_type}</dd>
<dt>Mínimo de noches</dt><dd>{minimum_nights}</dd>
<dt>Reseñas realizadas</dt><dd>{number_of_reviews}</dd>
</dl>
"""
apartamento_info = []
localizaciones = []

# Iterando con el dataframe convertido a diccionario (iterar sobre el dataframe mejor)
#dict_listing = lista_total.to_dict(orient='index')
#for a in dict_listing:
#    apartamento_info += [info_box_template.format(
#        id=dict_listing[a]['id'], name=dict_listing[a]['name'], price=dict_listing[a]['price'],
#        property_type=dict_listing[a]['property_type'], room_type=dict_listing[a]['room_type'],
#        minimum_nights=dict_listing[a]['minimum_nights'], number_of_reviews=dict_listing[a]['number_of_reviews'])]
#    localizaciones += [[dict_listing[a]['latitude'], dict_listing[a]['longitude']]]

# Iterando sobre el dataframe
for row in lista_total.itertuples():
    apartamento_info += [info_box_template.format(
        id=getattr(row, 'id'), name=getattr(row, 'name'), price=getattr(row, 'price'),
        property_type=getattr(row, 'property_type'), room_type=getattr(row, 'room_type'),
        minimum_nights=getattr(row, 'minimum_nights'), number_of_reviews=getattr(row, 'number_of_reviews'))]
    localizaciones += [[getattr(row, 'latitude'), getattr(row, 'longitude')]]

m = gmaps.Map()
marker_layer = gmaps.marker_layer(localizaciones, info_box_content=apartamento_info)
m.add_layer(marker_layer)
m

#interact

In [202]:
ruta_datasets = '../datasets/'
limit = 5 # numero de registros a leer, -1 para todos

fields_to_drop = ['host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'calculated_host_listings_count',
                  'availability_365', 'number_of_reviews', 'last_review', 'reviews_per_month']
listing = pd.read_csv(ruta_datasets + 'listings.csv').head(limit).drop(fields_to_drop, axis=1)

fields_to_drop = ['summary', 'space', 'description', 'neighborhood_overview', 'transit', 'access', 'interaction',
                  'name', 'notes', 'house_rules', 'host_about', 'host_verifications', 'latitude', 'longitude',
                  'room_type', 'minimum_nights']
listing_detail = pd.read_csv(ruta_datasets + 'listings.csv.gz',
                             dtype={'zipcode': str}).head(limit).drop(fields_to_drop, axis=1)

lista_total = pd.merge(listing, listing_detail, on='id')

# Obtener: precio minimo y maximo (para interact)
min_price = lista_total['price_x'].min(axis=0)
max_price = lista_total['price_x'].max(axis=0)

#print("min_price=%s, max_price=%s de %s registros" % (min_price, max_price, limit))
#sys.exit()

def mostrar_mapa(price_filtro_max):
    info_box_template = """
<dl>
<dt>ID: {id}</dt><dd>{name}</dd>
<dt>Precio</dt><dd>{price}</dd>
<dt>Tipo de propiedad y habitación</dt><dd>{property_type}, {room_type}</dd>
<dt>Mínimo de noches</dt><dd>{minimum_nights}</dd>
<dt>Reseñas realizadas</dt><dd>{number_of_reviews}</dd>
</dl>
"""
    apartamento_info = []
    localizaciones = []

    # Iterando sobre el dataframe
    for row in lista_total.itertuples():
        price_row = name=getattr(row, 'price_x')
        if price_row <= price_filtro_max:
            print("Si")
            apartamento_info += [info_box_template.format(
                id=getattr(row, 'id'), name=getattr(row, 'name'), price=getattr(row, 'price_y'),
                property_type=getattr(row, 'property_type'), room_type=getattr(row, 'room_type'),
                minimum_nights=getattr(row, 'minimum_nights'), number_of_reviews=getattr(row, 'number_of_reviews'))]
            localizaciones += [[getattr(row, 'latitude'), getattr(row, 'longitude')]]

    print("antes clean")
    clear_output()
    print("despues clean")
    m = gmaps.Map()
    marker_layer = gmaps.marker_layer(localizaciones, info_box_content=apartamento_info)
    m.add_layer(marker_layer)
    return m

#interact(mostrar_mapa, price = (min_price, max_price, 1))
interact_manual(mostrar_mapa, price_filtro_max = (min_price, max_price, 1))

<function __main__.mostrar_mapa>